In [10]:
# 📦 Step 1: Setup - Optimized for 16-core CPU
import os
import sys
import shutil
import random
import warnings
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import multiprocessing
import subprocess

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# ⚡ CPU Optimization: Enable all cores for TensorFlow (must be set BEFORE importing TF)
NUM_WORKERS = multiprocessing.cpu_count()
os.environ['TF_NUM_INTEROP_THREADS'] = str(NUM_WORKERS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(NUM_WORKERS)

# Install required packages
def install_if_missing(package, import_name=None):
    try:
        __import__(import_name or package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

install_if_missing("tqdm")
install_if_missing("scikit-learn", "sklearn")

from tqdm.auto import tqdm
import tensorflow as tf
import numpy as np

print(f"📊 TensorFlow: {tf.__version__}")
print(f"🧵 CPU cores: {NUM_WORKERS} (all will be used)")

# Check GPU (won't find any for AMD on Windows)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✅ GPU: {gpus}")
    BATCH_SIZE = 32
else:
    print("⚠️ AMD GPU not supported by TF on Windows - using optimized CPU mode")
    BATCH_SIZE = 32  # Larger batch OK with 16 cores

print(f"📦 Batch size: {BATCH_SIZE}")

📊 TensorFlow: 2.15.0
🧵 CPU cores: 16 (all will be used)
⚠️ AMD GPU not supported by TF on Windows - using optimized CPU mode
📦 Batch size: 32


In [ ]:
# 📁 Step 2: Paths & Configuration
BASE_DIR = Path(r"E:\FasalVaidya\backend\ml\models")
LOCAL_CACHE = BASE_DIR / "combined_balanced_dataset"
OUTPUT_DIR = BASE_DIR

DATA_SOURCES = [
    BASE_DIR / "Bigger CoLeaf DATASET" / "CoLeaf DATASET",
    BASE_DIR / "Propossed_Data" / "Contrast_Stretching",
    BASE_DIR / "Propossed_Data" / "Histogram_Equalization",
    BASE_DIR / "Propossed_Data" / "Log_Transformation",
    BASE_DIR / "Nitrogen deficiency",
    BASE_DIR / "ThorCam_semiFiltered",
    BASE_DIR / "POTASSIUM DEFICIENCY",
]

CLASS_MAPPING = {
    "healthy": "healthy", "control": "healthy", "-C": "healthy",
    "nitrogen-N": "nitrogen-N", "deficiency": "nitrogen-N", "N": "nitrogen-N",
    "phosphorus-P": "phosphorus-P", "-P": "phosphorus-P", "-P50": "phosphorus-P",
    "potasium-K": "potasium-K", "K": "potasium-K",
    "boron-B": "boron-B", "calcium-Ca": "calcium-Ca", "iron-Fe": "iron-Fe",
    "magnesium-Mg": "magnesium-Mg", "manganese-Mn": "manganese-Mn",
}

ALL_CLASSES = ["healthy", "nitrogen-N", "phosphorus-P", "potasium-K",
               "boron-B", "calcium-Ca", "iron-Fe", "magnesium-Mg", "manganese-Mn"]

# Hyperparameters - BATCH_SIZE set in Step 1 based on GPU availability
IMG_SIZE = 224
# BATCH_SIZE already set above (32 for GPU, 16 for CPU)
EPOCHS_PHASE1 = 25
EPOCHS_PHASE2 = 20
EPOCHS_PHASE3 = 15
TARGET_SAMPLES_PER_CLASS = 400

print(f"📁 Base: {BASE_DIR}")
print(f"⚙️ Batch size: {BATCH_SIZE}")
print(f"🔍 Checking datasets...")
for src in DATA_SOURCES:
    if src.exists():
        print(f"   ✅ {src.name}")
    else:
        print(f"   ❌ {src.name}")

📁 Base: E:\FasalVaidya\backend\ml\models
🔍 Checking datasets...
   ✅ CoLeaf DATASET
   ✅ Contrast_Stretching
   ✅ Histogram_Equalization
   ✅ Log_Transformation
   ✅ Nitrogen deficiency
   ✅ ThorCam_semiFiltered
   ✅ POTASSIUM DEFICIENCY


In [3]:
# 📂 Step 3: Combine & Balance Datasets
from sklearn.model_selection import train_test_split

def collect_images():
    class_images = {cls: [] for cls in ALL_CLASSES}
    
    for dataset_path in DATA_SOURCES:
        if not dataset_path.exists():
            continue
        name = dataset_path.name
        print(f"   📂 {name}")
        
        if "Nitrogen" in name or "nitrogen" in name:
            for split in ["train", "val", "test"]:
                split_dir = dataset_path / split
                if split_dir.exists():
                    for class_dir in split_dir.iterdir():
                        if class_dir.is_dir():
                            std = CLASS_MAPPING.get(class_dir.name)
                            if std in class_images:
                                for img in class_dir.glob("*"):
                                    if img.suffix.lower() in [".jpg", ".jpeg", ".png"]:
                                        class_images[std].append(img)
        elif "ThorCam" in name:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std = CLASS_MAPPING.get(class_dir.name)
                    if std in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png", ".tif", ".tiff"]:
                                class_images[std].append(img)
        elif "POTASSIUM" in name.upper():
            for img in dataset_path.glob("*"):
                if img.suffix.lower() in [".jpg", ".jpeg", ".png"]:
                    class_images["potasium-K"].append(img)
        else:
            for class_dir in dataset_path.iterdir():
                if class_dir.is_dir():
                    std = CLASS_MAPPING.get(class_dir.name, class_dir.name)
                    if std in class_images:
                        for img in class_dir.glob("*"):
                            if img.suffix.lower() in [".jpg", ".jpeg", ".png"]:
                                class_images[std].append(img)
    return class_images

print("📁 Scanning datasets...")
class_images = collect_images()

print("\n📊 Dataset stats:")
for cls in ALL_CLASSES:
    print(f"   {cls:15s}: {len(class_images[cls])}")

# Balance
print(f"\n⚖️ Balancing to {TARGET_SAMPLES_PER_CLASS}/class...")
for cls in ALL_CLASSES:
    n = len(class_images[cls])
    if n == 0:
        print(f"   ⚠️ {cls}: No samples!")
    elif n < TARGET_SAMPLES_PER_CLASS:
        class_images[cls].extend(random.choices(class_images[cls], k=TARGET_SAMPLES_PER_CLASS-n))
    elif n > TARGET_SAMPLES_PER_CLASS * 2:
        class_images[cls] = random.sample(class_images[cls], TARGET_SAMPLES_PER_CLASS)

# Copy to cache
if LOCAL_CACHE.exists():
    shutil.rmtree(LOCAL_CACHE)

def copy_file(args):
    src, dst = args
    try:
        shutil.copy2(src, dst)
        return 1
    except:
        return 0

tasks = []
for cls in ALL_CLASSES:
    (LOCAL_CACHE / cls).mkdir(parents=True, exist_ok=True)
    for i, src in enumerate(class_images[cls]):
        dst = LOCAL_CACHE / cls / f"{cls}_{i:05d}{src.suffix.lower()}"
        tasks.append((src, dst))

print(f"\n📁 Copying {len(tasks)} files...")
with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    results = list(tqdm(ex.map(copy_file, tasks), total=len(tasks)))
print(f"✅ Done!")

📁 Scanning datasets...
   📂 CoLeaf DATASET
   📂 Contrast_Stretching
   📂 Histogram_Equalization
   📂 Log_Transformation
   📂 Nitrogen deficiency
   📂 ThorCam_semiFiltered
   📂 POTASSIUM DEFICIENCY

📊 Dataset stats:
   healthy        : 1395
   nitrogen-N     : 523
   phosphorus-P   : 3508
   potasium-K     : 1815
   boron-B        : 401
   calcium-Ca     : 351
   iron-Fe        : 140
   magnesium-Mg   : 316
   manganese-Mn   : 266

⚖️ Balancing to 400/class...

📁 Copying 3724 files...


100%|██████████| 3724/3724 [00:37<00:00, 100.50it/s]

✅ Done!


In [4]:
# 🔀 Step 4: Create Splits
all_images, all_labels = [], []
class_to_idx = {c: i for i, c in enumerate(ALL_CLASSES)}

for cls in ALL_CLASSES:
    for img in (LOCAL_CACHE / cls).glob("*"):
        all_images.append(str(img))
        all_labels.append(class_to_idx[cls])

all_images, all_labels = np.array(all_images), np.array(all_labels)

train_imgs, test_imgs, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.1, stratify=all_labels, random_state=42)
train_imgs, val_imgs, train_labels, val_labels = train_test_split(
    train_imgs, train_labels, test_size=0.15, stratify=train_labels, random_state=42)

NUM_CLASSES = len(ALL_CLASSES)
print(f"✅ Train: {len(train_imgs)} | Val: {len(val_imgs)} | Test: {len(test_imgs)}")

✅ Train: 2848 | Val: 503 | Test: 373


In [5]:
# ⚡ Step 5: Data Pipeline
AUTOTUNE = tf.data.AUTOTUNE

def parse_image(path, label, training=True):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32) / 255.0
    if training:
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_brightness(img, 0.2)
        img = tf.image.random_contrast(img, 0.8, 1.2)
    return img, tf.one_hot(label, NUM_CLASSES)

def make_ds(imgs, labels, training=True):
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    if training:
        ds = ds.shuffle(len(imgs))
    ds = ds.map(lambda x, y: parse_image(x, y, training), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_ds(train_imgs, train_labels, True)
val_ds = make_ds(val_imgs, val_labels, False)
test_ds = make_ds(test_imgs, test_labels, False)
print(f"✅ Data pipeline ready")

✅ Data pipeline ready


In [6]:
# 🏗️ Step 6: Build Model
from sklearn.utils.class_weight import compute_class_weight

base = tf.keras.applications.EfficientNetV2S(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
print(f"✅ Model built: {model.count_params():,} params")

# Class weights
weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = dict(enumerate(weights))



✅ Model built: 21,125,993 params


In [7]:
# 🎯 Step 7: Callbacks
CHECKPOINT_DIR = OUTPUT_DIR / "checkpoints"
CHECKPOINT_DIR.mkdir(exist_ok=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint(str(CHECKPOINT_DIR / "best.keras"), monitor='val_accuracy', save_best_only=True),
]
print("✅ Callbacks ready")

✅ Callbacks ready


In [11]:
# 🚀 Step 8: Phase 1 - Train Head
print("="*50)
print("🚀 PHASE 1: Training Head (backbone frozen)")
print("="*50)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

h1 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_PHASE1,
               callbacks=callbacks, class_weight=class_weights)
print(f"✅ Phase 1 done! Val acc: {max(h1.history['val_accuracy']):.2%}")

🚀 PHASE 1: Training Head (backbone frozen)
Epoch 1/25
178/178 [==============================] - 134s 700ms/step - loss: 1.8985 - accuracy: 0.3427 - val_loss: 1.8875 - val_accuracy: 0.3797 - lr: 0.0010
Epoch 2/25
178/178 [==============================] - 117s 657ms/step - loss: 1.8389 - accuracy: 0.3722 - val_loss: 1.7494 - val_accuracy: 0.4493 - lr: 0.0010
Epoch 3/25
178/178 [==============================] - 119s 666ms/step - loss: 1.7667 - accuracy: 0.3947 - val_loss: 1.7086 - val_accuracy: 0.4553 - lr: 0.0010
Epoch 4/25
178/178 [==============================] - 115s 642ms/step - loss: 1.7189 - accuracy: 0.4199 - val_loss: 1.6389 - val_accuracy: 0.4632 - lr: 0.0010
Epoch 5/25
178/178 [==============================] - 111s 619ms/step - loss: 1.7125 - accuracy: 0.4315 - val_loss: 1.6564 - val_accuracy: 0.4453 - lr: 0.0010
Epoch 6/25
178/178 [==============================] - 110s 616ms/step - loss: 1.6848 - accuracy: 0.4477 - val_loss: 1.6338 - val_accuracy: 0.4612 - lr: 0.0010
Epo

In [12]:
# 🔥 Step 9: Phase 2 - Unfreeze Top 30%
print("="*50)
print("🔥 PHASE 2: Fine-tuning top 30%")
print("="*50)
      
base.trainable = True
for layer in base.layers[:int(len(base.layers) * 0.7)]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

h2 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_PHASE2,
               callbacks=callbacks, class_weight=class_weights)
print(f"✅ Phase 2 done! Val acc: {max(h2.history['val_accuracy']):.2%}")

🔥 PHASE 2: Fine-tuning top 30%
Epoch 1/20
178/178 [==============================] - 185s 946ms/step - loss: 1.5815 - accuracy: 0.4993 - val_loss: 1.6095 - val_accuracy: 0.4751 - lr: 1.0000e-04
Epoch 2/20
178/178 [==============================] - 167s 935ms/step - loss: 1.5162 - accuracy: 0.5235 - val_loss: 1.5096 - val_accuracy: 0.5189 - lr: 1.0000e-04
Epoch 3/20
178/178 [==============================] - 167s 935ms/step - loss: 1.5079 - accuracy: 0.5242 - val_loss: 1.5026 - val_accuracy: 0.5149 - lr: 1.0000e-04
Epoch 4/20
178/178 [==============================] - 166s 930ms/step - loss: 1.4850 - accuracy: 0.5320 - val_loss: 1.5107 - val_accuracy: 0.5010 - lr: 1.0000e-04
Epoch 5/20
178/178 [==============================] - 167s 933ms/step - loss: 1.4458 - accuracy: 0.5488 - val_loss: 1.4535 - val_accuracy: 0.5408 - lr: 1.0000e-04
Epoch 6/20
178/178 [==============================] - 167s 934ms/step - loss: 1.4520 - accuracy: 0.5499 - val_loss: 1.4734 - val_accuracy: 0.5268 - lr: 1.

In [13]:
# 🎓 Step 10: Phase 3 - Full Fine-tuning
print("="*50)
print("🎓 PHASE 3: Full fine-tuning")
print("="*50)

for layer in base.layers:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(2e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)
    
h3 = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_PHASE3,
               callbacks=callbacks, class_weight=class_weights)
print(f"✅ Phase 3 done! Val acc: {max(h3.history['val_accuracy']):.2%}")

🎓 PHASE 3: Full fine-tuning
Epoch 1/15
178/178 [==============================] - 511s 3s/step - loss: 1.2022 - accuracy: 0.6703 - val_loss: 1.1788 - val_accuracy: 0.7078 - lr: 2.0000e-05
Epoch 2/15
178/178 [==============================] - 512s 3s/step - loss: 1.1498 - accuracy: 0.7012 - val_loss: 1.1638 - val_accuracy: 0.6998 - lr: 2.0000e-05
Epoch 3/15
178/178 [==============================] - 440s 2s/step - loss: 1.1148 - accuracy: 0.7307 - val_loss: 1.1374 - val_accuracy: 0.7336 - lr: 2.0000e-05
Epoch 4/15
178/178 [==============================] - 433s 2s/step - loss: 1.1089 - accuracy: 0.7335 - val_loss: 1.1124 - val_accuracy: 0.7097 - lr: 2.0000e-05
Epoch 5/15
178/178 [==============================] - 435s 2s/step - loss: 1.0804 - accuracy: 0.7472 - val_loss: 1.1506 - val_accuracy: 0.7097 - lr: 2.0000e-05
Epoch 6/15
178/178 [==============================] - 434s 2s/step - loss: 1.0588 - accuracy: 0.7609 - val_loss: 1.1447 - val_accuracy: 0.7117 - lr: 2.0000e-05
Epoch 7/15
1

In [14]:
# 🧪 Step 11: Evaluate
print("="*50)
print("🧪 EVALUATION")
print("="*50)

loss, acc = model.evaluate(test_ds)
print(f"\n📊 Test Accuracy: {acc:.2%}")

# Quick classification report
from sklearn.metrics import classification_report
y_true, y_pred = [], []
for imgs, labels in test_ds:
    preds = model.predict(imgs, verbose=0)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=ALL_CLASSES, digits=3))

🧪 EVALUATION
24/24 [==============================] - 13s 519ms/step - loss: 0.9285 - accuracy: 0.8338

📊 Test Accuracy: 83.38%

📋 Classification Report:
              precision    recall  f1-score   support

     healthy      0.811     0.750     0.779        40
  nitrogen-N      0.833     0.755     0.792        53
phosphorus-P      0.857     0.750     0.800        40
  potasium-K      0.949     0.925     0.937        40
     boron-B      0.921     0.875     0.897        40
  calcium-Ca      0.818     0.900     0.857        40
     iron-Fe      0.921     0.875     0.897        40
magnesium-Mg      0.714     0.875     0.787        40
manganese-Mn      0.733     0.825     0.776        40

    accuracy                          0.834       373
   macro avg      0.840     0.837     0.836       373
weighted avg      0.840     0.834     0.834       373



In [15]:
# 💾 Step 12: Save Model
import json

model_path = OUTPUT_DIR / "plantvillage-npk-v3.h5"
model.save(str(model_path))
print(f"✅ Model saved: {model_path}")

with open(OUTPUT_DIR / "class_names.txt", 'w') as f:
    f.write('\n'.join(ALL_CLASSES))

config = {'model': 'EfficientNetV2-S', 'img_size': IMG_SIZE,
          'classes': ALL_CLASSES, 'test_accuracy': float(acc)}
with open(OUTPUT_DIR / "model_config.json", 'w') as f:
    json.dump(config, f, indent=2)

print(f"\n🎉 TRAINING COMPLETE!")
print(f"   Model: {model_path}")
print(f"   Accuracy: {acc:.2%}")
print(f"\n📋 Next: Update MODEL_PATH in tasks.json and restart backend")

✅ Model saved: E:\FasalVaidya\backend\ml\models\plantvillage-npk-v3.h5

🎉 TRAINING COMPLETE!
   Model: E:\FasalVaidya\backend\ml\models\plantvillage-npk-v3.h5
   Accuracy: 83.38%

📋 Next: Update MODEL_PATH in tasks.json and restart backend
